In [39]:
from nltk.corpus import stopwords
import string
import operator 
import json
from collections import Counter
import tweepy
from tweepy import OAuthHandler
import json
from nltk.tokenize import word_tokenize
from collections import defaultdict
import numpy as np

In [22]:
##function to preprocess text

import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [52]:
## make a list containing words we don't care
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'is', 'RT', '…', 'de', 'election', 'Election']

In [53]:
com = defaultdict(lambda : defaultdict(int))
n_docs = 0 ## counting number of twitters
with open('data/twitter_election.json', 'r') as f:
    count_stop_single = Counter()
    for line in f:
        if line.strip():
            n_docs += 1
            tweet = json.loads(line)
            terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
            terms_single = set(terms_stop)
            count_stop_single.update(terms_single)
            ## capture the co-occurrences words
            for i in range(len(terms_stop)-1):            
                for j in range(i+1, len(terms_stop)):
                    w1, w2 = sorted([terms_stop[i], terms_stop[j]])                
                    if w1 != w2:
                        com[w1][w2] += 1   

In [54]:
com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))
# Get the most frequent co-occurrences
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
print(terms_max[:10])

[(('@realDonaldTrump', 'Trump'), 281), (('Trump', 'Volunteer'), 269), (('Observer', 'Trump'), 269), (('Observer', 'Sign'), 269), (('Sign', 'Trump'), 269), (('Observer', 'Volunteer'), 268), (('Sign', 'Volunteer'), 268), (('Sign', 'today'), 268), (('#MakeAmericaGreatAgain', 'Trump'), 268), (('Trump', 'today'), 267)]


In [55]:
## build semantic orientation lists 
positive_vocab = []
negative_vocab = []
with open("positive-words.txt") as f:
    for line in f:
        line = line.strip('\n')
        positive_vocab.append(line)
with open("negative-words.txt") as f:
    for line in f:
        line = line.strip('\n')
        negative_vocab.append(line)

In [56]:
## counting the occurance frequency of the words 
p_t = {}
p_t_com = defaultdict(lambda : defaultdict(int))
 
for term, n in count_stop_single.items():
    p_t[term] = n / n_docs
    for t2 in com[term]:
        p_t_com[term][t2] = com[term][t2] / n_docs

In [57]:
## compute pointwise mutual information between words
pmi = defaultdict(lambda : defaultdict(int))
for t1 in p_t:
    for t2 in com[t1]:
        denom = p_t[t1] * p_t[t2]
        pmi[t1][t2] = math.log2(p_t_com[t1][t2] / denom)

semantic_orientation = {}
for term, n in p_t.items():
    positive_assoc = sum(pmi[term][tx] for tx in positive_vocab)
    negative_assoc = sum(pmi[term][tx] for tx in negative_vocab)
    semantic_orientation[term] = positive_assoc - negative_assoc

semantic_sorted = sorted(semantic_orientation.items(), 
                         key=operator.itemgetter(1), 
                         reverse=True)
top_pos = semantic_sorted[:10]
top_neg = semantic_sorted[-10:]
 
print("orientation for Trump: %f" % semantic_orientation['Trump'])
print("orientation for Clinton: %f" % semantic_orientation['Clinton'])

orientation for Trump: -79.921923
orientation for Clinton: -17.081703


### Trump and Clintion are both not so welcomed by the public, but it seems more people like Clinton